In [ ]:
!pip install tensorflow

In [ ]:
import time
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose, Reshape
from keras.layers import Flatten, BatchNormalization, Dense, Activation
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:

# below we create a data container for ready use of data from the specified path
def data_loader(path_of_data, size_of_batch, shape_of_image):
    data_container = ImageDataGenerator()
    data_container = data_container.flow_from_directory(
        path_of_data, target_size=(shape_of_image[0], shape_of_image[1]),
        batch_size=size_of_batch,
        class_mode=None)

    return data_container



def create_new_discriminator(shape_of_image):

    dis = Sequential()
    dis.add(Conv2D(filters=64, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',kernel_initializer='glorot_uniform',
                             input_shape=(shape_of_image)))
    dis.add(LeakyReLU(0.2))

    dis.add(Conv2D(filters=128, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    dis.add(BatchNormalization(momentum=0.5))
    dis.add(LeakyReLU(0.2))

    dis.add(Conv2D(filters=256, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    dis.add(BatchNormalization(momentum=0.5))
    dis.add(LeakyReLU(0.2))

    dis.add(Conv2D(filters=512, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    dis.add(BatchNormalization(momentum=0.5))
    dis.add(LeakyReLU(0.2))

    dis.add(Flatten())
    dis.add(Activation('sigmoid'))

    optimizer = Adam(lr=0.0002, beta_1=0.5)
    dis.compile(loss='binary_crossentropy',optimizer=optimizer)

    return dis



def create_new_generator():

    gen = Sequential()

    gen.add(Dense(units=4 * 4 * 512,kernel_initializer='glorot_uniform',input_shape=(1, 1, 100)))
    gen.add(Reshape(target_shape=(4, 4, 512)))
    gen.add(BatchNormalization(momentum=0.5))
    gen.add(Activation('relu'))

    gen.add(Conv2DTranspose(filters=256, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    gen.add(BatchNormalization(momentum=0.5))
    gen.add(Activation('relu'))

    gen.add(Conv2DTranspose(filters=128, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    gen.add(BatchNormalization(momentum=0.5))
    gen.add(Activation('relu'))

    gen.add(Conv2DTranspose(filters=64, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    gen.add(BatchNormalization(momentum=0.5))
    gen.add(Activation('relu'))

    gen.add(Conv2DTranspose(filters=3, kernel_size=(5, 5),strides=(2, 2), padding='same',data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    gen.add(Activation('tanh'))

    optimizer = Adam(lr=0.00015, beta_1=0.5)
    gen.compile(loss='binary_crossentropy',optimizer=optimizer)

    return gen


# Displays a figure of the generated images and saves them in as .png image
def save_generated_images(generated_images, epoch, batch_number):

    plt.figure(figsize=(8, 8), num=2)
    gs1 = gridspec.GridSpec(8, 8)
    gs1.update(wspace=0, hspace=0)

    for i in range(64):
        ax1 = plt.subplot(gs1[i])
        ax1.set_aspect('equal')
        image = generated_images[i, :, :, :]
        image += 1
        image *= 127.5
        fig = plt.imshow(image.astype(np.uint8))
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

    plt.tight_layout()
    save_name = 'generated images/generatedSamples_epoch' + str(
        epoch + 1) + '_batch' + str(batch_number + 1) + '.png'

    plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
    plt.pause(0.0000000001)
plt.show()

In [ ]:
# below is the driver function for training purpose
def trainfunction(size_of_batch, number_of_iterations, shape_of_image, path_of_data):
    #below we create an instance of generator and discriminator
    gen = create_new_generator()
    dis = create_new_discriminator(shape_of_image)

    gan = Sequential()
    # right now we are not training the discriminator only adding the instances of gen and dis to gan
    dis.trainable = False
    gan.add(gen)
    gan.add(dis)

    optimizer = Adam(learning_rate=0.00015, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer,
                metrics=['accuracy'])

    # below we crate a generator which will provide data as per need
    data_container = data_loader(path_of_data, size_of_batch, shape_of_image)

    # since the number of images in dataset is 11788 so number of batches will be as per given below
    batch_count = int(11788 / size_of_batch)

    # HERE WE RETURN THE ADVERSERIAL LOSS AND DISCRIMINATOR LOSS
    adversarial_loss = np.empty(shape=1)
    discriminator_loss = np.empty(shape=1)
    batches = np.empty(shape=1)

    # to update plot inside the loop
    plt.ion()

    curr_batch = 0 # starting batchnumber

    # below we do the actual training of gan model for number_of_iterations times
    d_loss = 0
    for epoch in range(number_of_iterations):

        print("Epoch " + str(epoch+1) + "/" + str(epoch) + " :")

        for batch_number in range(batch_count):

            start_time = time.time()

            # below we access the curr batch from the container i.e retieve it from the container and then normalize it
            real_images = data_container.next()
            real_images /= 127.5
            real_images -= 1

            current_batch_size = real_images.shape[0]
            print('curr batch size : ', current_batch_size)
            # creating random values that are to be feed to the discriminator
            noise = np.random.normal(0, 1,size=(current_batch_size,) + (1, 1, 100))

            # using generator to generate fake emages
            generated_images = gen.predict(noise)
            print('Generated : ' ,generated_images.shape)
            # here we will add some noise or inaccuracy to the labels of images that are to be feed to the discriminator
            real_y = (np.ones(current_batch_size) - np.random.random_sample(current_batch_size) * 0.2)
            fake_y = np.random.random_sample(current_batch_size) * 0.2
            print('Fake y : ', fake_y.shape)
            # below we train the discriminator
            dis.trainable = True

            #reshape of image
            #generated_images = np.reshape(generated_images, fake_y.shape)
            d_loss += dis.train_on_batch(generated_images, fake_y)
            d_loss += dis.train_on_batch(real_images, real_y)

            discriminator_loss = np.append(discriminator_loss, d_loss)

            # below we train the generator  so we will make discriminator untrainable
            dis.trainable = False

            noise = np.random.normal(0, 1,size=(current_batch_size * 2,) +(1, 1, 100))


            fake_y = (np.ones(current_batch_size * 2) - np.random.random_sample(current_batch_size * 2) * 0.2)

            g_loss = gan.train_on_batch(noise, fake_y)
            adversarial_loss = np.append(adversarial_loss, g_loss)
            batches = np.append(batches, current_batch)
            current_batch += 1


def main():
    path_of_data = 'CUB_200_2011/images/'
    size_of_batch = 64
    shape_of_image = (64, 64, 3)
    number_of_iterations = 400
    trainfunction(size_of_batch, number_of_iterations,
                shape_of_image, path_of_data)

if __name__ == "__main__":
  main()

Found 11788 images belonging to 200 classes.
Epoch 1/0 :
curr batch size :  64
2/2 [==============================] - 0s 11ms/step
Generated :  (64, 64, 64, 3)
Fake y :  (64,)


ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=21b938d78d4d40b7d6e55658d49a1e5da0a5a3c5a69621f2f2feda4bf05379e8
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import wget
#wget.download('http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz',)
#wget.download('https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz',)
wget.download('https://drive.google.com/open?id=0B0ywwgffWnLLcms2WWJQRFNSWXM&authuser=0',)

'open'

In [ ]:
import tarfile
tar = tarfile.open("102flowers.tgz")
tar.extractall()
tar.close()

EOFError: ignored

In [ ]:
!mkdir generated\ images